# ID2222 Data Mining, Homework 2
# **Discovery of Frequent Itemsets and Association Rules**

Brando Chiminelli, Tommaso Praturlon

November 21th, 2022

The goal of this notebook is to ...

## Import libraries and read the dataset

In order to run this notebook you need to import the dataset at this address (https://canvas.kth.se/courses/36211/files/5772174/download?wrap=1) in a 'data' directory.

In [8]:
import pandas as pd
import numpy as np
import random
import time
import matplotlib.pyplot as plt

PATH_TO_DATA = "../data/T10I4D100K.dat"
df_market = pd.read_csv(PATH_TO_DATA, header=None)
print("Data read successfully!")
# Delete duplicates from the dataset in the columns title and text

df_market.head()

Data read successfully!


,0
0,25 52 164 240 274 328 368 448 538 561 630 687 ...
1,39 120 124 205 401 581 704 814 825 834
2,35 249 674 712 733 759 854 950
3,39 422 449 704 825 857 895 937 954 964
4,15 229 262 283 294 352 381 708 738 766 853 883...


## Finding frequent itemsets with support at least s

Remind that an association rule is an implication X → Y, where X and Y are itemsets such that X∩Y=∅. Support of rule X → Y is the number of transactions that contain X⋃Y. Confidence of rule X → Y is the fraction of transactions containing X⋃Y in all transactions that contain X.

TASK

You are to solve the first sub-problem: to implement the A-Priori algorithm for finding frequent itemsets with support at least s in a dataset of sales transactions. Remind that support of an itemset is the number of transactions containing the itemset. To test and evaluate your implementation, write a program that uses your A-Priori algorithm implementation to discover frequent itemsets with support at least s in a given dataset of sales transactions.

In [9]:
pip install apyori

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5955 sha256=309a4c9cb213e7b476e016c019d2ca8188da5f21624615791df084885177ae9b
  Stored in directory: /Users/bralli/Library/Caches/pip/wheels/1a/4d/4f/a171c142b66180456b83a6180a70e68108127ea2d546ca174b
Successfully built apyori
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you a

In [12]:
S_SUPPORT = 0.01 

from apyori import apriori

items=[]
for i in range(0,20):
    items.append([df_market.values[i,j] for j in range(0,6)])

final_rule=apriori(items, min_support=0.1, min_confidence=0.5, min_lift=1.2, min_length=2)

final_results=list(final_rule)
print(final_rule)



IndexError: index 1 is out of bounds for axis 1 with size 1